# Problema 1

Implementar los siguientes métodos de descenso gradiente (naïve = tamaño de paso $\alpha$ constante):

- Descenso gradiente naïve con dirección de descenso aleatoria
- Descenso máximo naïve
- Descenso gradiente de Newton, con Hessiano exacto
- Un método de gradiente conjugado (Fletcher-Reeves, Hestenes-Stiefel, Polak-Ribière)
- Método BFGS

En cada uno de los métodos, su función debe recibir los siguientes argumentos:

- La función objetivo $f$.
- El gradiente de la función objetivo $df$.
- El hessiano $ddf$ (cuando sea necesario).
- Un punto inicial $x_0 \in \mathbb{R}^n$.
- El tamaño de paso $\alpha > 0$.
- El número máximo de iteraciones $maxIter$.
- La tolerancia $\varepsilon$, así como un criterio de paro.

Como resultado, sus algoritmos deben devolver: la mejor solución encontrada *best* (la última de las aproximaciones calculadas); la secuencia de iteraciones $x_k$; la secuencia de valores $f(x_k)$; la secuencia de errores en cada paso (según el error de su criterio de paro).

Además, es deseable indicar el número de iteraciones efectuadas por el algoritmo, y si se obtuvo o no convergencia del método.

In [ ]:
import numpy as np, time

## Descenso gradiente naïve con dirección de descenso aleatoria

In [ ]:
def norm(x, ord=None):
    if ord is None:
        ord = 2
    return np.linalg.norm(x, ord=ord)


Esta función la usaremos para calcular errores con norma midiendo la longitud del vector en un espacio vectorial.

In [ ]:
def projOrth(u, b_orth):
    v = u.copy().astype(float)
    if b_orth.ndim == 1:
        b = b_orth / (norm(b_orth) + 1e-15)
        v -= (v @ b) * b
    else:
        for b in b_orth:
            b = b / (norm(b) + 1e-15)
            v -= (v @ b) * b
    n = norm(v)
    if n < 1e-15:
        return v
    return v / n

Dado que tenemos el gradiente, debemos ir al negativo ($-\nabla f(x)$) de este para encontrar una válida dirección de descenso, pero también podemos tomar otras direccioens siempre que formen un ángulo menor de $90°$ con el gradiente.

Entonces para generar otras direcciones, se necesita un vector ortogonal al gradiente y combinarlo con un angulo $\phi$. Esta es la principal función de `projOrth`, con esta función vamos a recibir:

- `u`: punto de partida que luego se proyectará (vector).
- `b_orth`: esto es un vector al que se quiere ser ortogonal.

En caso que b tenga un valor de 1, indica que es un vector; si es mayor será una colección de vectores y queremos proyectar `u` a esto para que sea ortogonal a todos.

El proceso es que se normalizará del vector `b_orth` obteniendo un vector unitario `b` luego restamos al vector enviado a su proyección sobre `b` para asegurar que `v` sea ortogonal a `b`:

$$
v \leftarrow v - (v \cdot b) b
$$

Por último, debemos devolver un vector unitario ortogonal al `b_orth` pero si el vector `u` estaba casi alineado con `b_orth`, entonces al quitarle la proyección se queda en algo casi nulo ($\|v\| \approx 0$), y al normalizar esto ocurrirá un error numérica ($v / \|v\|$), por lo que debemos hacer una validación previa para retornar el vector:

   * Si $ \|v\| \approx 0$: devolvemos `v` sin normalizar (es decir, un vector casi nulo, aunque no sea unitario).
   * Si $ \|v\| > 0$: podemos normalizar con seguridad y devolvemos el vector unitario $v / \|v\|$.

> **Nota:** `v` es una copia del vector de partida `u` para no modificar el valor original enviado.

Esto lo utilizamos ya que las direcciones de descenso distintas al gradiente podemos tomar una dirección que forme un ángulo con el gradiente y para esto necesitamos:

$$
d = \cos(\phi)(-\hat g) + \sin(\phi) v
$$

donde:

* $-\hat g$ es el gradiente negativo unitario.
* $v$ es un vector unitario ortogonal al gradiente.

In [ ]:
def gradientDescentRandom(
    f, df, x0, alpha=0.1, maxIter=500, tol=1e-4, stopCrit="grad",
    normOrder=2, isPlottable=True, randomState=None, verbose=False, extra=None
):
    """
    Descenso con dirección de descenso aleatoria (o ángulo fijo).
    Retorna (best, xs, fxs, errors, metrics).
    """
    rng = np.random.default_rng(randomState)
    extra = extra or {}
    phiMode   = extra.get("phiMode", "random")
    phiFixed  = extra.get("phi", 0.0)
    phiRange  = extra.get("phiRange", (-np.pi/2, np.pi/2))

    x = np.array(x0, dtype=float).reshape(-1)
    n = x.size

    xs, fxs, errors = [x.copy()], [f(x)], []
    gradNorms, stepNorms, approxErrors, angles, dirs, xs2D, ks = [], [], [], [], [], [], []
    t0 = time.time()

    g = df(x)
    gradNorms.append(norm(g, normOrder))

    converged, stopReason = False, "maxIter"

    if verbose:
        print(f"[k=0] fx={fxs[-1]:.6e} | ||grad||={gradNorms[-1]:.6e} | x={x}")

    for k in range(1, maxIter+1):
        # Selección de ángulo
        if phiMode == "fixed":
            phi = float(phiFixed)
        else:
            lo, hi = phiRange
            phi = rng.uniform(lo, hi)

        # Dirección de descenso
        gnorm = norm(g, normOrder)
        if gnorm < 1e-15:
            d = -g
            angles.append(0.0)
        else:
            ghat = g / gnorm
            z = rng.normal(size=n)
            v = projOrth(z, ghat)
            if norm(v) < 1e-15:
                d = -ghat
                phi = 0.0
            else:
                d = np.cos(phi)*(-ghat) + np.sin(phi)*v

        # Paso naïve
        x_new = x + alpha * d
        fx_new = f(x_new)
        step = x_new - x

        # Error según stopCrit
        if stopCrit == "grad":
            err = norm(df(x_new), normOrder)
        elif stopCrit == "fx":
            err = abs(fx_new - fxs[-1])
        elif stopCrit == "xAbs":
            err = norm(step, normOrder)
        elif stopCrit == "xRel":
            err = norm(step, normOrder) / max(1.0, norm(x_new, normOrder))
        else:
            raise ValueError("stopCrit inválido.")

        # Registrar historia
        xs.append(x_new.copy()); fxs.append(fx_new); errors.append(err)
        gradNorms.append(norm(df(x_new), normOrder))
        stepNorms.append(norm(step, normOrder))
        approxErrors.append(err); angles.append(float(phi)); dirs.append(d.copy()); ks.append(k)
        if isPlottable and n == 2:
            xs2D.append(x_new.copy())

        if verbose:
            print(f"[k={k}] fx={fx_new:.6e} | err({stopCrit})={err:.6e} | "
                  f"||grad||={gradNorms[-1]:.6e} | ||step||={stepNorms[-1]:.6e} | phi={phi:.3f}")

        # Paro por tolerancia
        if err <= tol:
            converged, stopReason = True, "tolerance"
            x = x_new
            break

        # Avanzar
        x, g = x_new, df(x_new)

    timeSec = time.time() - t0
    best = x
    kstar = len(xs) - 1

    if verbose:
        print(f"==> stopReason={('tolerance' if converged else 'maxIter')} | "
              f"iters={kstar} | fx*={fxs[-1]:.6e} | ||grad*||={norm(df(best), normOrder):.6e}")

    metrics = {
        "method": "Gradient Descent (random direction)",
        "converged": converged,
        "stopReason": stopReason,
        "iterations": kstar,
        "finalX": best.copy(),
        "finalFx": fxs[-1],
        "gradNorm": norm(df(best), normOrder),
        "stepNorm": stepNorms[-1] if stepNorms else None,
        "approxError": errors[-1] if errors else norm(df(best), normOrder),
        "alpha": alpha,
        "timeSec": timeSec,
        "history": {
            "k": np.array(ks),
            "gradNorms": np.array(gradNorms),
            "stepNorms": np.array(stepNorms),
            "approxErrors": np.array(approxErrors),
            "angles": np.array(angles),
            "directions": np.array(dirs, dtype=float) if len(dirs) else None,
            "xs2D": np.array(xs2D) if xs2D else None,
        },
        "seed": randomState,
    }
    return best, np.array(xs), np.array(fxs), np.array(errors), metrics

## Descenso máximo naïve

## Descenso gradiente de Newton, con Hessiano exacto

## Gradiente conjugado (FR, HS o PR)

## Método BFGS

# Problema 2

- Testar sus algoritmos del Ejercicio 1.
- Para las funciones 2D, muestre visualizaciones de la secuencia de aproximaciones $\{x_k\}$ convergiendo al mínimo local de su función.

  ![Ejemplo Gráfica](../images/example_graph.png)

- En cada uno de los casos, hallar un tamaño de paso $\alpha$ que garantice la convergencia de los métodos, y elabore una tabla comparativa de los resultados, error, número de iteraciones requeridas por cada método. Por ejemplo:

  | Algoritmo de optimización    | Convergencia (Sí/No) | Número de Iteraciones | Solución | Error |
  | ---------------------------- | -------------------- | --------------------- | -------- | ----- |
  | Descenso gradiente           |                      |                       |          |       |
  | Descenso gradiente aleatorio |                      |                       |          |       |
  | Descenso máximo              |                      |                       |          |       |
  | Descenso de Newton           |                      |                       |          |       |
  | Fletcher-Reeves              |                      |                       |          |       |
  | BFGS                         |                      |                       |          |       |

- Elabore gráficas que muestren el error de aproximación, en función del número de iteración, y muestre la comparación de la evolución de la convergencia en sus cinco métodos. A partir de estas gráficas, discuta cuál de los métodos es más efectivo, en cada caso. Para ello, debe tomar en cuenta:
  - La solución aproximada obtenida
  - El error de aproximación
  - La norma del gradiente en la solución

## Inciso a

La función $f : \mathbb{R}^2 \to \mathbb{R}$, dada por

$$
f(x, y) = x^4 + y^4 - 4xy + \frac{1}{2}y + 1.
$$

Punto inicial: $x_0 = (-3, 1, -3, 1)^T$, Óptimo: $x^* = (-1.01463, -1.04453)^T, \; f(x^*) = -1.51132$.

## Inciso b

La función de Rosembrock 2-dimensional $f : \mathbb{R}^2 \to \mathbb{R}$, dada por

$$
f(x_1, x_2) = 100(x_2 - x_1^2)^2 + (1 - x_1)^2.
$$

Punto inicial: $x_0 = (-1.2, 1)^T$, Óptimo: $x^* = (1, 1)^T, \; f(x^*) = 0$.

## Inciso c

La función de Rosembrock 7-dimensional $f : \mathbb{R}^7 \to \mathbb{R}$, dada por

$$
f(x) = \sum_{i=1}^6 100(x_{i+1} - x_i^2)^2 + (1 - x_i)^2.
$$

Punto inicial: $x_0 = (-1.2, 1, 1, 1, 1, -1.2, 1)^T$, Óptimo: $x^* = (1, 1, \dots, 1)^T, \; f(x^*) = 0$.